In [1]:
# ✅ IMPORTS
import numpy as np
import cv2, os, pandas as pd, matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tqdm import tqdm  # opcional para progreso

# ✅ CARGAR MODELOS
meso = load_model("/kaggle/input/mesonet/tensorflow2/default/1/fine_tuned_meso.h5")
hamster = load_model("/kaggle/input/mesonet-original-hamster/mesonet_model.h5")  # ruta según cómo subas

print("Modelos cargados ✔️")

# ✅ RUTA DE VIDEO
video_path = "/kaggle/input/video.mp4"
assert os.path.exists(video_path)

# ✅ EXTRAE TODOS LOS FRAMES
def extract_all_frames(video_path):
    cap=cv2.VideoCapture(video_path); frames=[]
    while True:
        ret, frame = cap.read()
        if not ret: break
        frames.append(frame)
    cap.release()
    return frames

frames = extract_all_frames(video_path)

# ✅ ANALIZAR FRAMES CON AMBOS MODELOS
results = []
for i,frame in enumerate(frames):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = cv2.resize(rgb, (256,256))
    x = resized.astype(np.float32)/255.0
    x = np.expand_dims(x,0)

    pred_m = meso.predict(x)[0][0]
    pred_h = hamster.predict(x)[0][0]

    for name, pred in [("MesoNet", pred_m), ("Hamster", pred_h)]:
        label = "REAL" if pred < 0.5 else "FAKE"
        confidence = (1-pred) if label=="REAL" else pred
        results.append((i, name, label, confidence))

# ✅ TABLA DE RESULTADOS
df = pd.DataFrame(results, columns=["Frame","Model","Label","Confidence"])
display(df.groupby("Model").head(10).style.format({"Confidence":"{:.2%}"}))

# ✅ GUARDAR CSV
df.to_csv("compare_results.csv", index=False)

# ✅ RESUMEN GLOBAL
summary = df.groupby("Model").apply(lambda g: np.mean([r if l=="FAKE" else 1-r for _,_,l,r in g.values]))
print("\nResultado promedio por modelo:")
for model_name, score in summary.items():
    final = "REAL" if score < 0.5 else "FAKE"
    print(f"- {model_name}: {final} ({score:.2%} probabilidad de FAKE)")


2025-06-15 18:16:31.139048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750011391.402050      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750011391.480951      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-15 18:16:47.742380: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/input/mesonet-original-hamster/mesonet_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)